## 1. Preliminary

### 1.1 Context

Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près à la consommation et aux émissions des bâtiments non destinés à l’habitation.

Des relevés minutieux ont été effectués par les agents de la ville en 2016. Voici [les données](https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/Data_Scientist_P4/2016_Building_Energy_Benchmarking.csv) et [leur source](https://data.seattle.gov/dataset/2016-Building-Energy-Benchmarking/2bpz-gwpy). Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, **vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l’habitation** pour lesquels elles n’ont pas encore été mesurées.

<div class="alert alert-block alert-info">
Votre prédiction se basera sur les données structurelles des bâtiments (taille et usage des bâtiments, date de construction, situation géographique, ...)
</div>

Vous cherchez également à **évaluer l’intérêt de l’"[ENERGY STAR Score](https://www.energystar.gov/buildings/facility-owners-and-managers/existing-buildings/use-portfolio-manager/interpret-your-results/what)" pour la prédiction d’émissions**, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe. Vous l'intégrerez dans la modélisation et jugerez de son intérêt.
Vous sortez tout juste d’une réunion de brief avec votre équipe. Voici un récapitulatif de votre mission :
 1) Réaliser une courte analyse exploratoire.
 2) Tester différents modèles de prédiction afin de répondre au mieux à la problématique.

Avant de quitter la salle de brief, Douglas, le project lead, vous donne quelques pistes et erreurs à éviter :

> Douglas : L’objectif est de te passer des relevés de consommation annuels futurs (attention à la fuite de données). Nous ferons de toute façon pour tout nouveau bâtiment un premier relevé de référence la première année, donc rien ne t'interdit d’en déduire des variables structurelles aux bâtiments, par exemple la nature et proportions des sources d’énergie utilisées.. 
Fais bien attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations (peut-on déduire des choses intéressantes d’une simple adresse ?) et optimiser les performances en appliquant des transformations simples aux variables (normalisation, passage au log, etc.).
Mets en place une évaluation rigoureuse des performances de la régression, et optimise les hyperparamètres et le choix d’algorithmes de ML à l’aide d’une validation croisée.


### 1.2 Requirements

In [1]:
package_list = ("pandas", "numpy", "matplotlib", "seaborn", "scikit-learn", "mlflow")

In [2]:
!python3 -V

Python 3.10.13


In [3]:
txt = !python3 -m pip freeze
check = lambda i: any([(pack in i) for pack in package_list])
txt = [i for i in txt if check(i)]
txt

['matplotlib==3.8.2',
 'matplotlib-inline==0.1.6',
 'mlflow==2.11.3',
 'numpy @ file:///Users/runner/miniforge3/conda-bld/numpy_1704280780572/work/dist/numpy-1.26.3-cp310-cp310-macosx_11_0_arm64.whl#sha256=f96d0b051b72345dbc317d793b2b34c7c4b7f41b0b791ffc93e820c45ba6a91c',
 'pandas==2.2.0',
 'scikit-learn==1.4.0',
 'seaborn==0.13.2']

### 1.3 Imports

In [4]:
# built in
import os, warnings
import time

# data
import pandas as pd
import numpy as np

# metrics
from sklearn.metrics import mean_squared_error, r2_score

# estimators
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, Perceptron
from sklearn.neural_network import MLPRegressor

# model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

## mlflow
import mlflow
import mlflow.sklearn

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# exceptions
from sklearn.exceptions import ConvergenceWarning

# pipeline et preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import KNNImputer



### 1.4 Graphics and option

In [5]:
# warnings.filterwarnings('ignore)
warnings.filterwarnings(action='once')

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, module='_distutils_hack')
warnings.filterwarnings("ignore", category=DeprecationWarning, module='importlib')

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

<div class="alert alert-block alert-info">
We disable the warnings.</div>

### 1.5 Data

In [6]:
# !tree

In [7]:
# os.listdir()

In [8]:
path="./data/cleaned/"
filename="df_SiteEnergyUseWN_1.csv"

In [9]:
df = pd.read_csv(path+filename)
df.head()

,Neighborhood,GroupedPrimaryPropertyTypes,YearBuilt,NumberofBuildings,NumberofFloors,ENERGYSTARScore,DistanceToDowntown,proportion_surface_parking,proportion_surface_building,Log_SiteEnergyUseWN(kBtu)
0,DOWNTOWN,Bâtiments d'Hébergement,1927.0,1.0,12.0,60.0,0.864611,0.000000,1.000000,15.824652
1,DOWNTOWN,Bâtiments d'Hébergement,1996.0,1.0,11.0,61.0,0.907278,0.145453,0.854547,15.974742
2,DOWNTOWN,Bâtiments d'Hébergement,1969.0,1.0,41.0,43.0,1.047606,0.205748,0.794252,18.118725
3,DOWNTOWN,Bâtiments d'Hébergement,1926.0,1.0,10.0,56.0,1.038057,0.000000,1.000000,15.753792
4,DOWNTOWN,Bâtiments d'Hébergement,1980.0,1.0,18.0,75.0,1.100255,0.353115,0.646885,16.500395


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1379 entries, 0 to 1378
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Neighborhood                 1379 non-null   object 
 1   GroupedPrimaryPropertyTypes  1379 non-null   object 
 2   YearBuilt                    1379 non-null   float64
 3   NumberofBuildings            1379 non-null   float64
 4   NumberofFloors               1379 non-null   float64
 5   ENERGYSTARScore              924 non-null    float64
 6   DistanceToDowntown           1379 non-null   float64
 7   proportion_surface_parking   1379 non-null   float64
 8   proportion_surface_building  1379 non-null   float64
 9   Log_SiteEnergyUseWN(kBtu)    1379 non-null   float64
dtypes: float64(8), object(2)
memory usage: 107.9+ KB


## 2. Data Preparation

### 2.1 X & y

In [11]:
X = df.drop(columns="Log_SiteEnergyUseWN(kBtu)")
y = df['Log_SiteEnergyUseWN(kBtu)']

In [12]:
X.head()

,Neighborhood,GroupedPrimaryPropertyTypes,YearBuilt,NumberofBuildings,NumberofFloors,ENERGYSTARScore,DistanceToDowntown,proportion_surface_parking,proportion_surface_building
0,DOWNTOWN,Bâtiments d'Hébergement,1927.0,1.0,12.0,60.0,0.864611,0.000000,1.000000
1,DOWNTOWN,Bâtiments d'Hébergement,1996.0,1.0,11.0,61.0,0.907278,0.145453,0.854547
2,DOWNTOWN,Bâtiments d'Hébergement,1969.0,1.0,41.0,43.0,1.047606,0.205748,0.794252
3,DOWNTOWN,Bâtiments d'Hébergement,1926.0,1.0,10.0,56.0,1.038057,0.000000,1.000000
4,DOWNTOWN,Bâtiments d'Hébergement,1980.0,1.0,18.0,75.0,1.100255,0.353115,0.646885


In [13]:
y.head()

0    15.824652
1    15.974742
2    18.118725
3    15.753792
4    16.500395
Name: Log_SiteEnergyUseWN(kBtu), dtype: float64

## 3. Gradient boosting

In [14]:
# Set the MLflow experiment
mlflow.set_experiment('Gradient_Boosting_Energy_feature_engineering_2')

# Define the Gradient Boosting parameters
gb_params = {'n_estimators': 150, 'learning_rate': 0.1, 
             'max_depth': 2, 'min_samples_split': 20, 
             'min_samples_leaf': 10, 'max_features': 0.75}

# Results DataFrame
results = pd.DataFrame(columns=['Train R2', 'Test R2', 'Train MSE', 'Test MSE'])

# Perform multiple train-test splits
for split in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Preprocessing pipelines for numeric and categorical data
    numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X_train.select_dtypes(include=['object', 'category']).columns
    pipe_trans_num = Pipeline(steps=[('scaler', StandardScaler())])
    pipe_trans_cat = Pipeline(steps=[('ohe', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', pipe_trans_num, numeric_features),
            ('cat', pipe_trans_cat, categorical_features)
        ])

    # Complete pipeline
    k_val = 15
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("imputer", KNNImputer(n_neighbors=k_val)),
        ("model", GradientBoostingRegressor(**gb_params))
    ])

    with mlflow.start_run():
        # Train the pipeline
        pipeline.fit(X_train, y_train)

        # Predict and evaluate
        y_train_pred = pipeline.predict(X_train)
        y_test_pred = pipeline.predict(X_test)
        train_r2 = r2_score(y_train, y_train_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        train_mse = mean_squared_error(y_train, y_train_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)

        # Log parameters, metrics, and model
        mlflow.log_params(gb_params)
        mlflow.log_metric('Train R2', train_r2)
        mlflow.log_metric('Test R2', test_r2)
        mlflow.log_metric('Train MSE', train_mse)
        mlflow.log_metric('Test MSE', test_mse)
        mlflow.sklearn.log_model(pipeline, "model")

        # Store results
        results.loc[split] = [train_r2, test_r2, train_mse, test_mse]

# Save results to CSV
results.to_csv('gradient_boosting_energy_feature_eng_train_test_results.csv', index=False)


2024/04/24 18:50:51 INFO mlflow.tracking.fluent: Experiment with name 'Gradient_Boosting_Energy_feature_engineering_2' does not exist. Creating a new experiment.


In [15]:
# !mlflow ui